In [ ]:
from seagul.rl.algos import sac, ppo
from seagul.nn import MLP
from seagul.rl.models import SACModel, PPOModel 

input_size = 17
output_size = 6
layer_size = 64
num_layers = 2

policy = MLP(input_size, output_size*2, num_layers, layer_size)
value_fn = MLP(input_size, 1, num_layers, layer_size)
q1_fn = MLP(input_size + output_size, 1, num_layers, layer_size)
q2_fn = MLP(input_size + output_size, 1, num_layers, layer_size)
model = SACModel(policy, value_fn, q1_fn, q2_fn, 3)


ppo_policy = MLP(input_size, output_size, num_layers, layer_size)
ppo_model = PPOModel(ppo_policy, value_fn)

env_name = "HalfCheetah-v2"
model, rews, var_dict = sac(env_name, 100000, model, env_steps=1000, iters_per_update=100, min_steps_per_update=100, reward_stop=1000, exploration_steps=1000)
#model, rews, var_dict = ppo(env_name, 3e5, ppo_model)
globals().update(var_dict)









  0%|          | 0/100000 [00:00<?, ?it/s]

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.










  1%|          | 1000/100000 [00:00<00:31, 3157.87it/s]







  2%|▏         | 2000/100000 [00:01<00:49, 1984.28it/s]







  3%|▎         | 3000/100000 [00:14<07:03, 228.86it/s] 







  4%|▍         | 4000/100000 [00:28<11:19, 141.19it/s]







  4%|▍         | 4000/100000 [00:38<11:19, 141.19it/s]







  5%|▌         | 5000/100000 [00:41<14:20, 110.36it/s]







  6%|▌         | 6000/100000 [00:55<16:14, 96.45it/s] 







  6%|▌         | 6000/100000 [01:08<16:14, 96.45it/s]







  7%|▋         | 7000/100000 [01:08<17:32, 88.33it/s]







  8%|▊         | 8000/100000 [01:22<18:24, 83.29it/s]







  9%|▉         | 9000/100000 [01:35<18:53, 80.31it/s]







  9%|▉         | 9000/100000 [01:48<18:53, 80.31it/s]







 10%|█         | 10000/100000 [01:49<19:08, 78.34it/s]







 11%|█         | 11000/100000 [02:02<19:14, 77.06it/s]







 12%|█▏        | 12000/100000 [02:16<19:13, 76.29it/s]







 12%|█▏        | 12000/100000 [02:28<19:13, 76.29it/s]







 1

In [ ]:
import matplotlib.pyplot as plt

plt.plot(raw_rew_hist)
plt.figure()
plt.plot(pol_loss_hist)
plt.figure()
plt.plot(val_loss_hist)
plt.figure()
plt.plot(q1_loss_hist)
plt.plot(q2_loss_hist)


In [4]:
replay_obs1, replay_obs2, replay_acts, replay_rews, replay_done = replay_buf.sample_batch(replay_batch_size)


In [ ]:
replay_buf.sample_batch()

In [24]:
import gym
import torch
env = gym.make(env_name)
done = False
obs = env.reset()

obs_list = []
act_list = []
rew_list = []
avg_list = []
val_list = []

 
dtype = torch.float32
act_size = env.action_space.shape[0]
obs = env.reset()
done = False

while not done:
    model_obs = torch.as_tensor(obs, dtype=dtype).detach() #TODO.... (or not, this still works with ppo right??)
    obs_list.append(obs)
    
    noise = torch.randn(act_size)
    act, logp = model.select_action(model_obs.reshape(1, -1), noise)
    act = act.detach()

    obs, rew, done, _ = env.step(act.numpy().reshape(-1))
    env.render()

    act_list.append(act)
    rew_list.append(rew)

print(sum(rew_list))
plt.plot(obs_list)
plt.figure()
plt.plot(act_list)
plt.figure()
plt.plot(rew_list)
plt.figure()
plt.plot(val_list)
plt.figure()
plt.plot(avg_list)
    

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Creating window glfw


KeyboardInterrupt: 